In [ ]:
# IMPORT LIBRARIES

In [62]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.layers import BatchNormalization

In [2]:
!pip install fsspec


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# LOAD THE DATA1 FILE

In [63]:
d1 = pd.read_csv("D://network classification/UNSW_NB15_testing-set.csv")
d1.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [5]:
print(d1.shape)

(175341, 45)


In [64]:
# LOAD THE SECOND DATA FILE

In [65]:
d2 = pd.read_csv("D://network classification/UNSW_NB15_training-set.csv")
d2.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [66]:
print(d2.shape)

(82332, 45)


In [67]:
# CONCATENATE THE TWO DATA FILES 

In [68]:
d3 = pd.concat([d1,d2])
d3.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [69]:
print(d3.shape)

(257673, 45)


# PREPROCESSING THE DATA

In [70]:
X = d3.drop(columns = ['label'])
y = d3['label']

In [71]:
X.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,1,0,0,0,1,1,0,Normal
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,1,2,0,0,0,1,6,0,Normal
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,1,3,0,0,0,2,6,0,Normal
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,1,3,1,1,0,2,1,0,Normal
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,2,1,40,0,0,0,2,39,0,Normal


In [72]:
y.shape,X.shape

((257673,), (257673, 44))

In [73]:
print(list(d3))

['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label']


In [74]:
encoder = LabelEncoder()
X[['proto','service','state','attack_cat']]= X[['proto','service','state','attack_cat']].apply(encoder.fit_transform)

In [75]:
X.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,1,0.121478,113,0,4,6,4,258,172,74.087490,...,1,1,1,0,0,0,1,1,0,6
1,2,0.649902,113,0,4,14,38,734,42014,78.473372,...,1,1,2,0,0,0,1,6,0,6
2,3,1.623129,113,0,4,8,16,364,13186,14.170161,...,1,1,3,0,0,0,2,6,0,6
3,4,1.681642,113,3,4,12,12,628,770,13.677108,...,1,1,3,1,1,0,2,1,0,6
4,5,0.449454,113,0,4,10,6,534,268,33.373826,...,2,1,40,0,0,0,2,39,0,6


In [76]:
# splitting the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((206138, 44), (51535, 44), (206138,), (51535,))

In [78]:
X_train.shape[1]  # number of columns

44

In [79]:
# neural network structure
model = Sequential()
model.add(Dense(512,input_dim = 44, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [80]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 512)               23040     
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_16 (Dense)            (None, 128)               65664     
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 batch_normalization_2 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                     

In [81]:
# compiling  the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [90]:
history = model.fit(X_train,y_train,epochs = 90,verbose = 1,batch_size = 64).history

Epoch 1/90
3221/3221 [==============================] - 34s 11ms/step - loss: 0.4036 - accuracy: 0.8070
Epoch 2/90
3221/3221 [==============================] - 34s 10ms/step - loss: 0.3997 - accuracy: 0.8103
Epoch 3/90
3221/3221 [==============================] - 26s 8ms/step - loss: 0.3957 - accuracy: 0.8133
Epoch 4/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3943 - accuracy: 0.8131
Epoch 5/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3948 - accuracy: 0.8130
Epoch 6/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3925 - accuracy: 0.8139
Epoch 7/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3915 - accuracy: 0.8151
Epoch 8/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3915 - accuracy: 0.8153
Epoch 9/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3900 - accuracy: 0.8156
Epoch 10/90
3221/3221 [==============================] - 19s 6ms/step -

3221/3221 [==============================] - 19s 6ms/step - loss: 0.3547 - accuracy: 0.8231
Epoch 80/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3568 - accuracy: 0.8235
Epoch 81/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3553 - accuracy: 0.8241
Epoch 82/90
3221/3221 [==============================] - 20s 6ms/step - loss: 0.3579 - accuracy: 0.8228
Epoch 83/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3568 - accuracy: 0.8218
Epoch 84/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3623 - accuracy: 0.8232
Epoch 85/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3570 - accuracy: 0.8233
Epoch 86/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3548 - accuracy: 0.8235
Epoch 87/90
3221/3221 [==============================] - 19s 6ms/step - loss: 0.3534 - accuracy: 0.8237
Epoch 88/90
3221/3221 [==============================] - 19s 6ms/step - loss

In [93]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)

1611/1611 [==============================] - 4s 3ms/step


In [94]:
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

In [95]:
print(f'Accuracy: {accuracy}')

print(f'Confusion Matrix:\n{confusion}')

Accuracy: 0.8040360919763267
Confusion Matrix:
[[10650  7963]
 [ 2136 30786]]


In [96]:
model.save("network traffic classification.h5")

In [97]:
model.save_weights("network traffic classification save weights")